In [ ]:
### GPU configuration

! nvidia-smi

Tue Apr  4 14:58:51 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#!unzip -u "/content/drive/MyDrive/Deep Learning Algoritms/CNN/Breast Cancer Classification from BUSI/Breast Ultrasound Dataset.zip" -d "/content/drive/MyDrive/Deep Learning Algoritms/CNN/Breast Cancer Classification from BUSI/BUSI unzipped"

#print('\n\ndone!')

Archive:  /content/drive/MyDrive/Deep Learning Algoritms/CNN/Breast Cancer Classification from BUSI/Breast Ultrasound Dataset.zip
   creating: /content/drive/MyDrive/Deep Learning Algoritms/CNN/Breast Cancer Classification from BUSI/BUSI unzipped/Breast Ultrasound Dataset without GT/
   creating: /content/drive/MyDrive/Deep Learning Algoritms/CNN/Breast Cancer Classification from BUSI/BUSI unzipped/Breast Ultrasound Dataset without GT/benign/
  inflating: /content/drive/MyDrive/Deep Learning Algoritms/CNN/Breast Cancer Classification from BUSI/BUSI unzipped/Breast Ultrasound Dataset without GT/benign/benign (1).png  
  inflating: /content/drive/MyDrive/Deep Learning Algoritms/CNN/Breast Cancer Classification from BUSI/BUSI unzipped/Breast Ultrasound Dataset without GT/benign/benign (10).png  
  inflating: /content/drive/MyDrive/Deep Learning Algoritms/CNN/Breast Cancer Classification from BUSI/BUSI unzipped/Breast Ultrasound Dataset without GT/benign/benign (100).png  
  inflating: /co

In [ ]:
pip install split-folders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import splitfolders
import os


input_folder="/content/drive/MyDrive/Deep Learning Algoritms/CNN/Breast Cancer Classification from BUSI/BUSI unzipped/Resized_Breast Ultrasound Dataset without GT"


output="/content/drive/MyDrive/Deep Learning Algoritms/CNN/Breast Cancer Classification from BUSI/BUSI splitted"



splitfolders.ratio(input_folder, output, seed=42, ratio=(.7,.1,.2)) ### train 70%, val 10%, test 20%


Copying files: 780 files [00:14, 53.38 files/s]


In [ ]:
####### transfer learning model

################# import various transfer elarning models ###################

#from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.vgg16 import VGG16
#from tensorflow.keras.applications.vgg19 import VGG19
#from tensorflow.keras.applications import Xception

#############################################################################

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob

import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger

import tensorflow as tf


########
IMAGE_SIZE = [224,224]
CLASS=3
######


#mod = InceptionV3(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)
mod = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)
#mod = DenseNet201(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)
#mod = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)
#mod = Xception(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

for layer in mod.layers:
    layer.trainable = False

x = Flatten()(mod.output)

prediction = Dense(CLASS, activation='softmax')(x)

model = Model(inputs=mod.input, outputs=prediction)

############################ optimizer and learning rate ##################

adam = tf.keras.optimizers.Adam(lr=0.001)
model.compile(
  loss='categorical_crossentropy', ### for multi class calsification
  #loss='binary_crossentropy', ### for binary calsification
  optimizer=adam,
  metrics=['accuracy']
)

#########################################################################


model.optimizer.get_config()
print("\n\n")
model.summary()

58900480/58889256 [==============================] - 0s 0us/step



Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                           

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
### image datagenerator

train_datagen = ImageDataGenerator(shear_range = 0.2,
                                   zoom_range = 0.2,
                                   #brightness_range=(0.4, 0.7),
                                   #width_shift_range=0.2,
                                   #height_shift_range=0.2,
                                   vertical_flip= True,
                                   horizontal_flip = True

                                   )

val_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/Deep Learning Algoritms/CNN/Breast Cancer Classification from BUSI/BUSI splitted/train',  
                                                 target_size = (224,224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

val_set = val_datagen.flow_from_directory('/content/drive/MyDrive/Deep Learning Algoritms/CNN/Breast Cancer Classification from BUSI/BUSI splitted/val',
                                            target_size = (224,224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

test_datagen = ImageDataGenerator(rescale = 1./255)

test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/Deep Learning Algoritms/CNN/Breast Cancer Classification from BUSI/BUSI splitted/test',
                                            target_size = (224,224),
                                            batch_size = 32,
                                            class_mode = 'categorical')





Found 545 images belonging to 3 classes.
Found 77 images belonging to 3 classes.
Found 158 images belonging to 3 classes.


In [ ]:
from PIL import Image

filepath = "/content/drive/MyDrive/Deep Learning Algoritms/CNN/Breast Cancer Classification from BUSI/h5/vgg16_bus.h5"


checkpoint1 = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,save_weights_only=True,
                             save_best_only=True, mode='max')

log_csv = CSVLogger('/content/drive/MyDrive/Deep Learning Algoritms/CNN/Breast Cancer Classification from BUSI/csv/vgg16_bus.csv', separator=',', append=False)
    
    
callbacks_list = [checkpoint1,log_csv]


r = model.fit_generator(
    training_set,
    epochs=100,
    validation_data=val_set,
    steps_per_epoch = len(training_set),
    validation_steps=len(val_set),
    callbacks=callbacks_list,
    shuffle=False
)



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/100
18/18 [==============================] - ETA: 0s - loss: 9.3918 - accuracy: 0.4826
Epoch 1: val_accuracy improved from -inf to 0.27273, saving model to /content/drive/MyDrive/Deep Learning Algoritms/CNN/Breast Cancer Classification from BUSI/h5/vgg16_bus.h5
18/18 [==============================] - 24s 621ms/step - loss: 9.3918 - accuracy: 0.4826 - val_loss: 1.5792 - val_accuracy: 0.2727
Epoch 2/100
17/18 [===========================>..] - ETA: 0s - loss: 4.3685 - accuracy: 0.6213
Epoch 2: val_accuracy improved from 0.27273 to 0.55844, saving model to /content/drive/MyDrive/Deep Learning Algoritms/CNN/Breast Cancer Classification from BUSI/h5/vgg16_bus.h5
18/18 [==============================] - 9s 509ms/step - loss: 4.3930 - accuracy: 0.6202 - val_loss: 0.9308 - val_accuracy: 0.5584
Epoch 3/100
17/18 [===========================>..] - ETA: 0s - loss: 3.6200 - accuracy: 0.7114
Epoch 3: val_accuracy improved from 0.55844 to 0.58442, saving model to /content/drive/MyDrive/Deep

In [ ]:
#### testing model

model.load_weights('/content/drive/MyDrive/Deep Learning Algoritms/CNN/Breast Cancer Classification from BUSI/h5/vgg16_bus.h5')  ### load weights from h5 file

preds = model.evaluate_generator(test_set)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  """


Loss = 0.7843225598335266
Test Accuracy = 0.6202531456947327
